# Quantamental Database Explorer

Explore your Parquet-based data storage:
- **Parquet Files** (`data/factors/`) - All historical data (prices, factors, macro)
- **DuckDB** - SQL query interface over Parquet files

**No SQLite** - We use Parquet as the single source of truth for efficient analytics!


In [1]:
# Setup: imports and configuration
import sys
from pathlib import Path

import duckdb
import pandas as pd
from IPython.display import display

# Configure pandas to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Resolve paths (works from repo root or notebooks/)
CWD = Path.cwd().resolve()
CANDIDATES_ROOT = [CWD, CWD.parent]

for root in CANDIDATES_ROOT:
    if (root / "config" / "settings.py").exists():
        if str(root) not in sys.path:
            sys.path.insert(0, str(root))
        break

def resolve_first_existing(paths: list[Path], fallback: Path) -> Path:
    for p in paths:
        if p.exists():
            return p
    return fallback

PARQUET_DIR = resolve_first_existing(
    [root / "data" / "factors" for root in CANDIDATES_ROOT],
    CWD / "data" / "factors",
)

DUCKDB_PATH = resolve_first_existing(
    [root / "data" / "factors" / "factors.duckdb" for root in CANDIDATES_ROOT],
    CWD / "data" / "factors" / "factors.duckdb",
)

print(f"📁 Parquet Dir: {PARQUET_DIR}")
print(f"🦆 DuckDB: {DUCKDB_PATH}")
print(f"✅ Parquet dir exists: {PARQUET_DIR.exists()}")
print(f"✅ DuckDB exists: {DUCKDB_PATH.exists()}")


📁 Parquet Dir: /Users/andres/Downloads/Cursor/quant/data/factors
🦆 DuckDB: /Users/andres/Downloads/Cursor/quant/data/factors/factors.duckdb
✅ Parquet dir exists: True
✅ DuckDB exists: True


---
## 1. Parquet Files Overview
### Available Datasets


In [2]:
# List all Parquet files in the factors directory
import os

parquet_files = [f for f in os.listdir(PARQUET_DIR) if f.endswith('.parquet')]

print("Available Parquet Files:")
print("=" * 80)

file_info = []
for file in sorted(parquet_files):
    path = PARQUET_DIR / file
    size_mb = path.stat().st_size / (1024 * 1024)
    file_info.append({
        'File': file,
        'Size (MB)': f"{size_mb:.2f}",
        'Path': str(path)
    })

df_files = pd.DataFrame(file_info)
display(df_files)


Available Parquet Files:


,File,Size (MB),Path
0,factors_all.parquet,183.71,/Users/andres/Downloads/Cursor/quant/data/fact...
1,factors_price.parquet,183.71,/Users/andres/Downloads/Cursor/quant/data/fact...
2,macro.parquet,0.24,/Users/andres/Downloads/Cursor/quant/data/fact...
3,macro_z.parquet,1.01,/Users/andres/Downloads/Cursor/quant/data/fact...
4,prices.parquet,20.28,/Users/andres/Downloads/Cursor/quant/data/fact...


---
## 2. Prices Dataset
### Wide Format: Date × Symbols


In [3]:
# prices.parquet
prices_path = PARQUET_DIR / "prices.parquet"

if prices_path.exists():
    df_prices = pd.read_parquet(prices_path)
    
    print("=" * 80)
    print("prices.parquet - Wide format (date × tickers)")
    print("=" * 80)
    print(f"Shape: {df_prices.shape[0]:,} dates × {df_prices.shape[1]} tickers")
    print(f"Index: {df_prices.index.name}")
    print(f"Date range: {df_prices.index.min()} to {df_prices.index.max()}")
    print(f"\nFirst 10 tickers: {list(df_prices.columns[:10])}")
    print(f"Last 10 tickers: {list(df_prices.columns[-10:])}")
    
    print("\nFirst 5 rows:")
    display(df_prices.head(5))
    
    print("\nLast 5 rows:")
    display(df_prices.tail(5))
else:
    print(f"⚠️  prices.parquet not found at {prices_path}")


prices.parquet - Wide format (date × tickers)
Shape: 25,521 dates × 504 tickers
Index: date
Date range: 1927-12-30 00:00:00-05:00 to 2025-10-24 00:00:00-04:00

First 10 tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']
Last 10 tickers: ['WTW', 'WDAY', 'WYNN', 'XEL', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZTS', '^GSPC']

First 5 rows:


,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,AON,APA,APO,AAPL,AMAT,APP,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BAX,BDX,BRK-B,BBY,TECH,BIIB,BLK,BX,XYZ,BK,BA,BKNG,BSX,BMY,AVGO,BR,BRO,BF-B,BLDR,BG,BXP,CHRW,CDNS,CPT,CPB,COF,CAH,KMX,CCL,CARR,CAT,CBOE,CBRE,CDW,COR,CNC,CNP,CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,COIN,CL,CMCSA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CPAY,CTVA,CSGP,COST,CTRA,CRWD,CCI,CSX,CMI,CVS,DHR,DRI,DDOG,DVA,DAY,DECK,DE,DELL,DAL,DVN,DXCM,FANG,DLR,DG,DLTR,D,DPZ,DASH,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,EME,EMR,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ERIE,ESS,EL,EG,EVRG,ES,EXC,EXE,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FITB,FSLR,FE,FI,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GE,GEHC,GEV,GEN,GNRC,GD,GIS,GM,GPC,GILD,GPN,GL,GDDY,GS,HAL,HIG,HAS,HCA,DOC,HSIC,HSY,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUBB,HUM,HBAN,HII,IBM,IEX,IDXX,ITW,INCY,IR,PODD,INTC,IBKR,ICE,IFF,IP,IPG,INTU,ISRG,IVZ,INVH,IQV,IRM,JBHT,JBL,JKHY,J,JNJ,JCI,JPM,K,KVUE,KDP,KEY,KEYS,KMB,KIM,KMI,KKR,KLAC,KHC,KR,LHX,LH,LRCX,LW,LVS,LDOS,LEN,LII,LLY,LIN,LYV,LKQ,LMT,L,LOW,LULU,LYB,MTB,MPC,MAR,MMC,MLM,MAS,MA,MTCH,MKC,MCD,MCK,MDT,MRK,META,MET,MTD,MGM,MCHP,MU,MSFT,MAA,MRNA,MHK,MOH,TAP,MDLZ,MPWR,MNST,MCO,MS,MOS,MSI,MSCI,NDAQ,NTAP,NFLX,NEM,NWSA,NWS,NEE,NKE,NI,NDSN,NSC,NTRS,NOC,NCLH,NRG,NUE,NVDA,NVR,NXPI,ORLY,OXY,ODFL,OMC,ON,OKE,ORCL,OTIS,PCAR,PKG,PLTR,PANW,PSKY,PH,PAYX,PAYC,PYPL,PNR,PEP,PFE,PCG,PM,PSX,PNW,PNC,POOL,PPG,PPL,PFG,PG,PGR,PLD,PRU,PEG,PTC,PSA,PHM,PWR,QCOM,DGX,RL,RJF,RTX,O,REG,REGN,RF,RSG,RMD,RVTY,HOOD,ROK,ROL,ROP,ROST,RCL,SPGI,CRM,SBAC,SLB,STX,SRE,NOW,SHW,SPG,SWKS,SJM,SW,SNA,SOLV,SO,LUV,SWK,SBUX,STT,STLD,STE,SYK,SMCI,SYF,SNPS,SYY,TMUS,TROW,TTWO,TPR,TRGP,TGT,TEL,TDY,TER,TSLA,TXN,TPL,TXT,TMO,TJX,TKO,TTD,TSCO,TT,TDG,TRV,TRMB,TFC,TYL,TSN,USB,UBER,UDR,ULTA,UNP,UAL,UPS,URI,UNH,UHS,VLO,VTR,VLTO,VRSN,VRSK,VZ,VRTX,VTRS,VICI,V,VST,VMC,WRB,GWW,WAB,WMT,DIS,WBD,WM,WAT,WEC,WFC,WELL,WST,WDC,WY,WSM,WMB,WTW,WDAY,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS,^GSPC
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1927-12-30 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Last 5 rows:


,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,AON,APA,APO,AAPL,AMAT,APP,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BAX,BDX,BRK-B,BBY,TECH,BIIB,BLK,BX,XYZ,BK,BA,BKNG,BSX,BMY,AVGO,BR,BRO,BF-B,BLDR,BG,BXP,CHRW,CDNS,CPT,CPB,COF,CAH,KMX,CCL,CARR,CAT,CBOE,CBRE,CDW,COR,CNC,CNP,CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,COIN,CL,CMCSA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CPAY,CTVA,CSGP,COST,CTRA,CRWD,CCI,CSX,CMI,CVS,DHR,DRI,DDOG,DVA,DAY,DECK,DE,DELL,DAL,DVN,DXCM,FANG,DLR,DG,DLTR,D,DPZ,DASH,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,EME,EMR,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ERIE,ESS,EL,EG,EVRG,ES,EXC,EXE,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FITB,FSLR,FE,FI,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GE,GEHC,GEV,GEN,GNRC,GD,GIS,GM,GPC,GILD,GPN,GL,GDDY,GS,HAL,HIG,HAS,HCA,DOC,HSIC,HSY,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUBB,HUM,HBAN,HII,IBM,IEX,IDXX,ITW,INCY,IR,PODD,INTC,IBKR,ICE,IFF,IP,IPG,INTU,ISRG,IVZ,INVH,IQV,IRM,JBHT,JBL,JKHY,J,JNJ,JCI,JPM,K,KVUE,KDP,KEY,KEYS,KMB,KIM,KMI,KKR,KLAC,KHC,KR,LHX,LH,LRCX,LW,LVS,LDOS,LEN,LII,LLY,LIN,LYV,LKQ,LMT,L,LOW,LULU,LYB,MTB,MPC,MAR,MMC,MLM,MAS,MA,MTCH,MKC,MCD,MCK,MDT,MRK,META,MET,MTD,MGM,MCHP,MU,MSFT,MAA,MRNA,MHK,MOH,TAP,MDLZ,MPWR,MNST,MCO,MS,MOS,MSI,MSCI,NDAQ,NTAP,NFLX,NEM,NWSA,NWS,NEE,NKE,NI,NDSN,NSC,NTRS,NOC,NCLH,NRG,NUE,NVDA,NVR,NXPI,ORLY,OXY,ODFL,OMC,ON,OKE,ORCL,OTIS,PCAR,PKG,PLTR,PANW,PSKY,PH,PAYX,PAYC,PYPL,PNR,PEP,PFE,PCG,PM,PSX,PNW,PNC,POOL,PPG,PPL,PFG,PG,PGR,PLD,PRU,PEG,PTC,PSA,PHM,PWR,QCOM,DGX,RL,RJF,RTX,O,REG,REGN,RF,RSG,RMD,RVTY,HOOD,ROK,ROL,ROP,ROST,RCL,SPGI,CRM,SBAC,SLB,STX,SRE,NOW,SHW,SPG,SWKS,SJM,SW,SNA,SOLV,SO,LUV,SWK,SBUX,STT,STLD,STE,SYK,SMCI,SYF,SNPS,SYY,TMUS,TROW,TTWO,TPR,TRGP,TGT,TEL,TDY,TER,TSLA,TXN,TPL,TXT,TMO,TJX,TKO,TTD,TSCO,TT,TDG,TRV,TRMB,TFC,TYL,TSN,USB,UBER,UDR,ULTA,UNP,UAL,UPS,URI,UNH,UHS,VLO,VTR,VLTO,VRSN,VRSK,VZ,VRTX,VTRS,VICI,V,VST,VMC,WRB,GWW,WAB,WMT,DIS,WBD,WM,WAT,WEC,WFC,WELL,WST,WDC,WY,WSM,WMB,WTW,WDAY,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS,^GSPC
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-10-20 00:00:00-04:00,154.779999,69.809998,129.509995,232.059998,242.179993,343.399994,240.559998,14.40,108.769997,143.000000,254.210007,127.209999,74.250000,96.139999,76.730003,134.929993,177.970001,68.589996,193.789993,256.549988,257.019989,64.400002,216.479996,8.15,105.690002,117.820000,349.480011,77.690002,193.080002,145.429993,472.679993,187.389999,303.329987,127.669998,246.220001,344.109985,22.490000,125.190002,262.239990,228.130005,565.940002,83.400002,86.970001,63.189999,146.479996,279.260010,209.149994,26.100000,177.740005,307.950012,283.029999,3968.570068,187.050003,160.250000,683.130005,46.279999,48.709999,52.040001,23.129999,191.000000,494.079987,81.010002,62.540001,146.630005,1160.000000,160.440002,76.500000,108.500000,216.820007,5141.879883,100.529999,43.590000,349.239990,230.679993,87.519997,28.930000,122.459999,97.250000,70.669998,127.459999,329.640015,103.089996,31.020000,214.869995,157.270004,43.029999,28.860001,57.669998,531.179993,237.089996,159.880005,155.380005,327.390015,35.610001,40.220001,83.930000,184.139999,94.949997,251.509995,154.479996,41.630001,268.200012,88.080002,306.839996,154.899994,192.330002,70.650002,99.300003,50.290001,119.589996,267.619995,75.089996,68.440002,66.699997,343.779999,78.750000,29.570000,18.500000,87.059998,101.720001,141.229996,370.000000,74.989998,44.759998,85.279999,286.049988,61.980000,74.959999,936.1099

---
## 3. Price Factors
### Momentum, Volatility, Beta


In [4]:
# factors_price.parquet
factors_price_path = PARQUET_DIR / "factors_price.parquet"

if factors_price_path.exists():
    df_factors_price = pd.read_parquet(factors_price_path)
    
    print("=" * 80)
    print("factors_price.parquet - Price-based factors (long format)")
    print("=" * 80)
    print(f"Shape: {df_factors_price.shape[0]:,} rows × {df_factors_price.shape[1]} columns")
    print(f"Index: {df_factors_price.index.names}")
    print(f"Columns: {list(df_factors_price.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_factors_price.head(5))
    
    print("\nLast 5 rows:")
    display(df_factors_price.tail(5))
    
    print("\nDescriptive statistics:")
    display(df_factors_price.describe())
else:
    print(f"⚠️  factors_price.parquet not found at {factors_price_path}")


factors_price.parquet - Price-based factors (long format)
Shape: 12,862,584 rows × 5 columns
Index: ['date', 'symbol']
Columns: ['mom_12_1', 'mom_6_1', 'mom_3_1', 'vol_60d', 'beta_60d']

First 5 rows:


factor                            mom_12_1  mom_6_1  mom_3_1  vol_60d  \
date                      symbol                                        
1927-12-30 00:00:00-05:00 A            NaN      NaN      NaN      NaN   
                          AAPL         NaN      NaN      NaN      NaN   
                          ABBV         NaN      NaN      NaN      NaN   
                          ABNB         NaN      NaN      NaN      NaN   
                          ABT          NaN      NaN      NaN      NaN   

factor                            beta_60d  
date                      symbol            
1927-12-30 00:00:00-05:00 A            NaN  
                          AAPL         NaN  
                          ABBV         NaN  
                          ABNB         NaN  
                          ABT          NaN


Last 5 rows:


factor                            mom_12_1   mom_6_1   mom_3_1   vol_60d  \
date                      symbol                                           
2025-10-24 00:00:00-04:00 YUM     0.113777  0.011609  0.034336  0.215673   
                          ZBH    -0.127302 -0.050610  0.022604  0.262526   
                          ZBRA   -0.238917  0.214377 -0.091382  0.425017   
                          ZTS    -0.177476 -0.096466 -0.067656  0.182486   
                          ^GSPC   0.117187  0.183597  0.037747  0.105533   

factor                            beta_60d  
date                      symbol            
2025-10-24 00:00:00-04:00 YUM     0.357566  
                          ZBH     0.649332  
                          ZBRA    2.574511  
                          ZTS     0.580128  
                          ^GSPC   1.000000


Descriptive statistics:


factor,mom_12_1,mom_6_1,mom_3_1,vol_60d,beta_60d
count,4.513908e+06,4.513908e+06,4.513908e+06,4.494252e+06,4.494252e+06
mean,1.464351e-01,6.475349e-02,2.574320e-02,3.092704e-01,9.294694e-01
std,4.680424e-01,2.688549e-01,1.633981e-01,2.766116e-01,6.819370e-01
min,-1.080000e+01,-1.227273e+01,-3.857143e+00,0.000000e+00,-1.426598e+01
25%,-6.203470e-02,-5.903661e-02,-4.680904e-02,1.948205e-01,5.774021e-01
50%,9.729257e-02,4.860993e-02,2.163142e-02,2.597469e-01,9.001270e-01
75%,2.811482e-01,1.625237e-01,9.160962e-02,3.622984e-01,1.222648e+00
max,5.218276e+01,3.252308e+01,2.539286e+01,5.398988e+01,1.282943e+02


---
## 4. All Factors Combined
### Price + Fundamental Factors


In [5]:
# factors_all.parquet
factors_all_path = PARQUET_DIR / "factors_all.parquet"

if factors_all_path.exists():
    df_factors_all = pd.read_parquet(factors_all_path)
    
    print("=" * 80)
    print("factors_all.parquet - All factors (price + fundamentals)")
    print("=" * 80)
    print(f"Shape: {df_factors_all.shape[0]:,} rows × {df_factors_all.shape[1]} columns")
    print(f"Index: {df_factors_all.index.names}")
    print(f"Columns: {list(df_factors_all.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_factors_all.head(5))
    
    print("\nLast 5 rows:")
    display(df_factors_all.tail(5))
else:
    print(f"⚠️  factors_all.parquet not found at {factors_all_path}")


factors_all.parquet - All factors (price + fundamentals)
Shape: 12,862,584 rows × 5 columns
Index: ['date', 'symbol']
Columns: ['mom_12_1', 'mom_6_1', 'mom_3_1', 'vol_60d', 'beta_60d']

First 5 rows:


factor                            mom_12_1  mom_6_1  mom_3_1  vol_60d  \
date                      symbol                                        
1927-12-30 00:00:00-05:00 A            NaN      NaN      NaN      NaN   
                          AAPL         NaN      NaN      NaN      NaN   
                          ABBV         NaN      NaN      NaN      NaN   
                          ABNB         NaN      NaN      NaN      NaN   
                          ABT          NaN      NaN      NaN      NaN   

factor                            beta_60d  
date                      symbol            
1927-12-30 00:00:00-05:00 A            NaN  
                          AAPL         NaN  
                          ABBV         NaN  
                          ABNB         NaN  
                          ABT          NaN


Last 5 rows:


factor                            mom_12_1   mom_6_1   mom_3_1   vol_60d  \
date                      symbol                                           
2025-10-24 00:00:00-04:00 YUM     0.113777  0.011609  0.034336  0.215673   
                          ZBH    -0.127302 -0.050610  0.022604  0.262526   
                          ZBRA   -0.238917  0.214377 -0.091382  0.425017   
                          ZTS    -0.177476 -0.096466 -0.067656  0.182486   
                          ^GSPC   0.117187  0.183597  0.037747  0.105533   

factor                            beta_60d  
date                      symbol            
2025-10-24 00:00:00-04:00 YUM     0.357566  
                          ZBH     0.649332  
                          ZBRA    2.574511  
                          ZTS     0.580128  
                          ^GSPC   1.000000

---
## 5. Macroeconomic Indicators
### CPI, Unemployment, Fed Funds, Yields


In [6]:
# macro.parquet
macro_path = PARQUET_DIR / "macro.parquet"

if macro_path.exists():
    df_macro = pd.read_parquet(macro_path)
    
    print("=" * 80)
    print("macro.parquet - Macroeconomic indicators")
    print("=" * 80)
    print(f"Shape: {df_macro.shape[0]:,} days × {df_macro.shape[1]} indicators")
    print(f"Index: {df_macro.index.name}")
    print(f"Date range: {df_macro.index.min()} to {df_macro.index.max()}")
    print(f"Columns: {list(df_macro.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro.tail(5))
    
    print("\nDescriptive statistics:")
    display(df_macro.describe())
else:
    print(f"⚠️  macro.parquet not found at {macro_path}")


macro.parquet - Macroeconomic indicators
Shape: 20,563 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2025-10-24 00:00:00
Columns: ['cpi_yoy', 'unrate', 'fed_funds', 'dgs10', 't10y2y']

First 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
2025-10-20,0.030227,4.3,4.22,4.00,0.54
2025-10-21,0.030227,4.3,4.22,3.98,0.53
2025-10-22,0.030227,4.3,4.22,3.97,0.52
2025-10-23,0.030227,4.3,4.22,4.01,0.53
2025-10-24,0.030227,4.3,4.22,4.01,0.54



Descriptive statistics:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
count,20302.000000,20302.000000,18607.000000,16649.000000,12889.000000
mean,0.035314,5.665989,4.623278,5.828356,0.850566
std,0.028718,1.709143,3.575337,2.944766,0.919778
min,-0.029881,2.500000,0.050000,0.520000,-2.410000
25%,0.016667,4.300000,1.930000,3.880000,0.180000
50%,0.028681,5.500000,4.320000,5.480000,0.800000
75%,0.044118,6.700000,6.170000,7.550000,1.480000
max,0.145892,14.800000,19.100000,15.840000,2.910000


---
## 6. Z-Scored Macro Indicators
### Standardized for Cross-Sectional Analysis


In [7]:
# macro_z.parquet
macro_z_path = PARQUET_DIR / "macro_z.parquet"

if macro_z_path.exists():
    df_macro_z = pd.read_parquet(macro_z_path)
    
    print("=" * 80)
    print("macro_z.parquet - Standardized (z-scored) macro indicators")
    print("=" * 80)
    print(f"Shape: {df_macro_z.shape[0]:,} days × {df_macro_z.shape[1]} indicators")
    print(f"Index: {df_macro_z.index.name}")
    print(f"Date range: {df_macro_z.index.min()} to {df_macro_z.index.max()}")
    print(f"Columns: {list(df_macro_z.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro_z.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro_z.tail(5))
    
    print("\nDescriptive statistics (should be ~0 mean, ~1 std):")
    display(df_macro_z.describe())
else:
    print(f"⚠️  macro_z.parquet not found at {macro_z_path}")


macro_z.parquet - Standardized (z-scored) macro indicators
Shape: 20,563 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2025-10-24 00:00:00
Columns: ['macro_z_cpi_yoy', 'macro_z_unrate', 'macro_z_fed_funds', 'macro_z_dgs10', 'macro_z_t10y2y']

First 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
2025-10-20,-0.712691,0.135288,0.482680,0.572451,0.568060
2025-10-21,-0.713817,0.138233,0.481511,0.554217,0.553789
2025-10-22,-0.714945,0.141204,0.480343,0.544475,0.539579
2025-10-23,-0.716075,0.144203,0.479173,0.577426,0.554770
2025-10-24,-0.717208,0.147230,0.478004,0.576227,0.569954



Descriptive statistics (should be ~0 mean, ~1 std):


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
count,20198.000000,20198.000000,18503.000000,16545.000000,12785.000000
mean,0.100571,-0.050640,0.285049,0.061540,-0.217255
std,1.345490,1.485897,1.487612,1.427846,1.361473
min,-3.716056,-2.331360,-4.265090,-4.875493,-3.712167
25%,-0.767100,-1.257918,-0.958755,-0.947379,-1.360420
50%,-0.163997,-0.382143,0.323762,-0.081291,-0.396647
75%,1.062566,0.903086,1.433669,1.187607,0.922475
max,4.621561,16.756748,6.764902,5.303855,4.099746


---
## 7. DuckDB Query Examples
### SQL Interface Over Parquet Files


In [8]:
# Connect to DuckDB and run SQL queries
if DUCKDB_PATH.exists():
    con = duckdb.connect(str(DUCKDB_PATH))
    
    print("=" * 80)
    print("DuckDB Query Examples")
    print("=" * 80)
    
    # Example 1: Get AAPL prices for recent dates
    print("\n1. Recent AAPL prices:")
    query1 = """
    SELECT * FROM prices 
    WHERE symbol = 'AAPL' 
    ORDER BY date DESC 
    LIMIT 10
    """
    # Note: prices is in wide format, so this won't work as-is
    # Let's query factors instead
    
    # Example 2: Get factors for AAPL
    print("\n2. Recent AAPL factors:")
    query2 = """
    SELECT * FROM factors_price 
    WHERE symbol = 'AAPL' 
    ORDER BY date DESC 
    LIMIT 10
    """
    try:
        result = con.sql(query2).df()
        display(result)
    except Exception as e:
        print(f"Query failed: {e}")
    
    # Example 3: Top momentum stocks
    print("\n3. Top 10 momentum stocks (most recent date):")
    query3 = """
    WITH latest_date AS (
        SELECT MAX(date) as max_date FROM factors_price
    )
    SELECT symbol, mom_12_1, mom_6_1, mom_3_1
    FROM factors_price
    WHERE date = (SELECT max_date FROM latest_date)
    AND mom_12_1 IS NOT NULL
    ORDER BY mom_12_1 DESC
    LIMIT 10
    """
    try:
        result = con.sql(query3).df()
        display(result)
    except Exception as e:
        print(f"Query failed: {e}")
    
    con.close()
else:
    print(f"⚠️  DuckDB not found at {DUCKDB_PATH}")


DuckDB Query Examples

1. Recent AAPL prices:

2. Recent AAPL factors:
Query failed: IO Error: No files found that match the pattern "data/factors/factors_price.parquet"

3. Top 10 momentum stocks (most recent date):
Query failed: IO Error: No files found that match the pattern "data/factors/factors_price.parquet"


---
## Summary

**Parquet-Based Architecture:**
- **Single Source of Truth**: All data in Parquet files (no SQLite)
- **Wide Format**: prices.parquet (date × symbols)
- **Long Format**: factors (date, symbol, factors)
- **DuckDB**: SQL query layer (no storage, just views)

**Incremental Updates:**
- Run `python scripts/update_daily.py` to fetch new data
- Run `python scripts/add_symbol.py TICKER` to add stocks
- Cron job runs daily at 6 PM automatically


In [9]:
# Summary statistics
print("=" * 80)
print("DATA SUMMARY")
print("=" * 80)

summary = []

for file in ['prices', 'factors_price', 'factors_all', 'macro', 'macro_z']:
    path = PARQUET_DIR / f"{file}.parquet"
    if path.exists():
        df = pd.read_parquet(path)
        size_mb = path.stat().st_size / (1024 * 1024)
        
        summary.append({
            'Dataset': file,
            'Rows': f"{df.shape[0]:,}",
            'Columns': df.shape[1],
            'Size (MB)': f"{size_mb:.2f}",
            'Date Range': f"{df.index.get_level_values(0).min() if isinstance(df.index, pd.MultiIndex) else df.index.min()} to {df.index.get_level_values(0).max() if isinstance(df.index, pd.MultiIndex) else df.index.max()}"
        })

df_summary = pd.DataFrame(summary)
display(df_summary)

print("\n✅ All datasets ready for quantamental research!")
print("📚 See INCREMENTAL_UPDATES.md for architecture details")
print("🔄 Run 'python scripts/update_daily.py' to update data")


DATA SUMMARY


,Dataset,Rows,Columns,Size (MB),Date Range
0,prices,"25,521",504,20.28,1927-12-30 00:00:00-05:00 to 2025-10-24 00:00:...
1,factors_price,"12,862,584",5,183.71,1927-12-30 00:00:00-05:00 to 2025-10-24 00:00:...
2,factors_all,"12,862,584",5,183.71,1927-12-30 00:00:00-05:00 to 2025-10-24 00:00:...
3,macro,"20,563",5,0.24,1947-01-01 00:00:00 to 2025-10-24 00:00:00
4,macro_z,"20,563",5,1.01,1947-01-01 00:00:00 to 2025-10-24 00:00:00



✅ All datasets ready for quantamental research!
📚 See INCREMENTAL_UPDATES.md for architecture details
🔄 Run 'python scripts/update_daily.py' to update data


### `macro.parquet` - Macroeconomic indicators


In [10]:
# macro.parquet
macro_path = PARQUET_DIR / "macro.parquet"

if macro_path.exists():
    df_macro = pd.read_parquet(macro_path)
    
    print("=" * 80)
    print("macro.parquet - Macroeconomic indicators")
    print("=" * 80)
    print(f"Shape: {df_macro.shape[0]:,} days × {df_macro.shape[1]} indicators")
    print(f"Index: {df_macro.index.name}")
    print(f"Date range: {df_macro.index.min()} to {df_macro.index.max()}")
    print(f"Columns: {list(df_macro.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro.tail(5))
    
    print("\nDescriptive statistics:")
    display(df_macro.describe())
else:
    print(f"⚠️  macro.parquet not found at {macro_path}")


macro.parquet - Macroeconomic indicators
Shape: 20,563 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2025-10-24 00:00:00
Columns: ['cpi_yoy', 'unrate', 'fed_funds', 'dgs10', 't10y2y']

First 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
date,,,,,
2025-10-20,0.030227,4.3,4.22,4.00,0.54
2025-10-21,0.030227,4.3,4.22,3.98,0.53
2025-10-22,0.030227,4.3,4.22,3.97,0.52
2025-10-23,0.030227,4.3,4.22,4.01,0.53
2025-10-24,0.030227,4.3,4.22,4.01,0.54



Descriptive statistics:


,cpi_yoy,unrate,fed_funds,dgs10,t10y2y
count,20302.000000,20302.000000,18607.000000,16649.000000,12889.000000
mean,0.035314,5.665989,4.623278,5.828356,0.850566
std,0.028718,1.709143,3.575337,2.944766,0.919778
min,-0.029881,2.500000,0.050000,0.520000,-2.410000
25%,0.016667,4.300000,1.930000,3.880000,0.180000
50%,0.028681,5.500000,4.320000,5.480000,0.800000
75%,0.044118,6.700000,6.170000,7.550000,1.480000
max,0.145892,14.800000,19.100000,15.840000,2.910000


### `macro_z.parquet` - Z-scored macro indicators


In [11]:
# macro_z.parquet
macro_z_path = PARQUET_DIR / "macro_z.parquet"

if macro_z_path.exists():
    df_macro_z = pd.read_parquet(macro_z_path)
    
    print("=" * 80)
    print("macro_z.parquet - Standardized (z-scored) macro indicators")
    print("=" * 80)
    print(f"Shape: {df_macro_z.shape[0]:,} days × {df_macro_z.shape[1]} indicators")
    print(f"Index: {df_macro_z.index.name}")
    print(f"Date range: {df_macro_z.index.min()} to {df_macro_z.index.max()}")
    print(f"Columns: {list(df_macro_z.columns)}")
    
    print("\nFirst 5 rows:")
    display(df_macro_z.head(5))
    
    print("\nLast 5 rows:")
    display(df_macro_z.tail(5))
    
    print("\nDescriptive statistics (should be ~0 mean, ~1 std):")
    display(df_macro_z.describe())
else:
    print(f"⚠️  macro_z.parquet not found at {macro_z_path}")


macro_z.parquet - Standardized (z-scored) macro indicators
Shape: 20,563 days × 5 indicators
Index: date
Date range: 1947-01-01 00:00:00 to 2025-10-24 00:00:00
Columns: ['macro_z_cpi_yoy', 'macro_z_unrate', 'macro_z_fed_funds', 'macro_z_dgs10', 'macro_z_t10y2y']

First 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1947-01-02,NaN,NaN,NaN,NaN,NaN
1947-01-03,NaN,NaN,NaN,NaN,NaN
1947-01-06,NaN,NaN,NaN,NaN,NaN
1947-01-07,NaN,NaN,NaN,NaN,NaN



Last 5 rows:


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
date,,,,,
2025-10-20,-0.712691,0.135288,0.482680,0.572451,0.568060
2025-10-21,-0.713817,0.138233,0.481511,0.554217,0.553789
2025-10-22,-0.714945,0.141204,0.480343,0.544475,0.539579
2025-10-23,-0.716075,0.144203,0.479173,0.577426,0.554770
2025-10-24,-0.717208,0.147230,0.478004,0.576227,0.569954



Descriptive statistics (should be ~0 mean, ~1 std):


,macro_z_cpi_yoy,macro_z_unrate,macro_z_fed_funds,macro_z_dgs10,macro_z_t10y2y
count,20198.000000,20198.000000,18503.000000,16545.000000,12785.000000
mean,0.100571,-0.050640,0.285049,0.061540,-0.217255
std,1.345490,1.485897,1.487612,1.427846,1.361473
min,-3.716056,-2.331360,-4.265090,-4.875493,-3.712167
25%,-0.767100,-1.257918,-0.958755,-0.947379,-1.360420
50%,-0.163997,-0.382143,0.323762,-0.081291,-0.396647
75%,1.062566,0.903086,1.433669,1.187607,0.922475
max,4.621561,16.756748,6.764902,5.303855,4.099746


---
## Summary

**Database Architecture:**
- **SQLite** (`stock_data.db`) - Operational cache for recent data
- **Parquet** files - Analytics-optimized historical datasets
  - `prices.parquet` - Wide format (date × tickers)
  - `factors_price.parquet` - Long format (date, symbol, factors)
  - `factors_all.parquet` - Combined price + fundamental factors
  - `macro.parquet` - Raw macroeconomic indicators
  - `macro_z.parquet` - Standardized macro indicators
